#MGSC662 - Final Project
#Authors: Ram Babu, Alfonso Cabello, Arnaud Guzman-Annès, Jules Zielinski
#Date: November 21, 2020

In [1]:
# Import libraries

import pulp as plp
from pulp import *
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
# Data - Aircrafts

# Aicrafts
aircrafts = ["Boeing 787","Airbus 320","Boeing 737","Boeing 767","Boeing 777"]

# Availability
aircrafts_avail = [48, 97, 136, 54, 94]

# Maximum seating capacity for each aircraft
seat = [300, 150, 160, 260, 320]




'''
#aircrafts
0- Boeing 787 - PAX = 300, OPC = 5310, Availability = 48
1- Airbus 320 - PAX = 150, OPC = 2840, Availability = 97
2- Boeing 737 - PAX = 160, OPC = 3010, Availability = 136
3- Boeing 767 - PAX = 260, OPC = 4980, Availability = 54
4- Boeing 777 - PAX = 320, OPC = 7220, Availability = 94
'''

##########################################################

# Data - Flights

# Airports (Hubs)
airports = ['ORD','DEN','IAH','LAX','EWR','SFO','IAD']

# Fares for each flight
fare = np.array([
[0  ,	47 ,	81 ,	50 ,	136,	116,	123],
[51 ,	0  ,	64 ,	47 ,	137,	70 ,	156],
[79 ,	79 ,	0  ,	94 ,	68 ,	215,	189],
[45 ,	59 ,	94 ,	0  ,	129,	64 ,	156],
[125,	125,	79 ,	139,	0  ,	124,	149],
[110,	70 ,	209,	59 ,	139,	0  ,	307],
[119,	149,	279,	125,	139,	299,	0  ]])


# Time per flight (in hrs)
time_flight = np.array([
[0   ,	2.55,	2.5 ,	4.3 ,	2.2 ,	4.5 ,	1.75],
[2.55,	0   ,	2.4 ,	2.3 ,	3.6 ,	2.55,	3.25],
[2.5 ,	2.4 ,	0   ,	3.4 ,	3.5 ,	4.45,	2.8 ],
[4.3 ,	2.3 ,	3.4 ,	0   ,	4.9 ,	1.4 ,	4.75],
[2.2 ,	3.6 ,	3.5 ,	4.9 ,	0   ,	6.25,	1.25],
[4.5 ,	2.55,	4.45,	1.4 ,	6.25,	0   ,	5   ],
[1.75,	3.25,	2.8 ,	4.75,	1.25,	5   ,	0   ]]) 

# Frequency of flights per day
frequency = np.array([
[0  ,	10 ,	5  ,	6  ,	12 ,	4  ,	4  ],
[9  ,	0  ,	5  ,	5  ,	6  ,	6  ,	4  ],
[4  ,	4  ,	0  ,	10 ,	10 ,	5  ,	5  ],
[5  ,	4  ,	9  ,	0  ,	15 ,	15 ,	10 ],
[11 ,	5  ,	9  ,	14 ,	0  ,	12 ,	11 ],
[3  ,	5  ,	4  ,	14 ,	11 ,	0  ,	5  ],
[3  ,	3  ,	4  ,	9  ,	10 ,	4  ,	0  ]])

'''
#Hubs
0- Chicago–O'Hare - ORD
1- Denver - DEN
2- Houston–Intercontinental - IAH
3- Los Angeles - LAX
4- Newark - EWR
5- San Francisco - SFO
6- Washington–Dulles - IAD
'''


"\n#Hubs\n0- Chicago–O'Hare - ORD\n1- Denver - DEN\n2- Houston–Intercontinental - IAH\n3- Los Angeles - LAX\n4- Newark - EWR\n5- San Francisco - SFO\n6- Washington–Dulles - IAD\n"

In [3]:
def run_prob():
    prob = plp.LpProblem("United_Airlines_flight")
    prob.sense = LpMaximize
    
    # Cost per flight hour for each aircraft
    cost_flight_hour = [5310, 2840, 3010, 4980, 7220]
    
    #Adding variable cost
    #Assumption: Variable cost has a normal distribution around fixed cost for each aircraft type and a standard deviation of 1000
    
    for i in range(len(aircrafts)):
        cost_flight_hour[i] = cost_flight_hour[i] + np.random.normal()*1000
        
    # Passenger Demand
    demand = np.array([
    [0   ,	1900,	650 ,	1100,	2350,	1000,	400 ],
    [1800,	0   ,	600 ,	1500,	1000,	1000,	500 ],
    [750 ,	550 ,	0   ,	1300,	1350,	500,	950 ],
    [1200,	1600,	1200,	0   ,	2100,	3000,	1950],
    [2450,	900 ,	1250,	2000,	0   ,	2150,	1900],
    [1100,	1200,	450 ,	2800,	2200,	0   ,	750 ],
    [450 ,	550 ,	900 ,	1900,	1850,	950 ,	0   ]]) 
        
    
    #Stochastic Demand
    #Assumption: Demand has a normal distribution around forecasted demand for each route and a standard deviation of 100
    for i in range(len(airports)):
        for j in range(len(airports)):
            if i!=j:
                demand[i][j] = demand[i][j]+np.random.normal()*100
    
    # Variables
    x = {i:{j:{k: plp.LpVariable(name="{} to {},type {}".format(airports[i],airports[j],aircrafts[k]), lowBound = 0, cat = plp.LpInteger)
          for k in range(len(aircrafts))}
        for j in range(len(airports))}
    for i in range(len(airports))}
    
    # Revenue = Σ Fare ij * Passanger demand ij 

    Revenue = pulp.lpSum(fare[i][j] * demand[i][j]
                       for i in range(len(time_flight))
                       for j in range(len(fare)))
    
    #Objective function

    # Profit (Z) = Revenue - Cost
    prob += Revenue - pulp.lpSum(x[i][j][k] * cost_flight_hour[k] * time_flight[i][j]  
                       for j in range(len(airports))
                       for k in range(len(cost_flight_hour))
                       for i in range(len(time_flight)))
    
    #Contraints - Part 1

    # No flight from/to the same departure point

    for i in range(len(airports)):
        for k in range(len(aircrafts)):
            prob += (x[i][i][k] == 0)
            
    #Contraints - Part 2 - Supply of Seats

    # We need to meet the demand of passengers moving from airport to airport

    # Σ(X ijk)(Sk) ≥ (demand from i to j)

    for i in range(len(airports)):
        for j in range(len(airports)):
            prob += pulp.lpSum(seat[k] * x[i][j][k] for k in range(len(aircrafts))) >= demand[i][j]
            
    

    #Contraints - Part 3 - Aircraft Availability

    # Here we are assuming we can run our aircraft for up to 20 hours in a day

    # 20 * Σ X ijk ≥ (Freq ij)(Time ij)

    for i in range(len(airports)):
        for j in range(len(airports)):
            prob += pulp.lpSum(20 * x[i][j][k] for k in range(len(aircrafts))) >= frequency[i][j] * time_flight[i][j]
            
    
    #Contraints - Part 4 - Flight Frequency

    # We want each route to be covered a certain number of times

    # Σ Xijk ≥ frequency ij

    for i in range(len(airports)):
        for j in range(len(airports)):
            prob += pulp.lpSum(x[i][j][k] for k in range(len(aircrafts))) >= frequency[i][j]
            
    
    # Constraint - Part 5 – Aircraft Availability

    for k in range(len(aircrafts)):
        prob += pulp.lpSum(pulp.lpSum(x[i][j][k] for i in range(len(airports)))for j in range(len(airports))) <= aircrafts_avail[k]
        
    
    prob.solve()
    total_aircrafts = []
    sum=0
    for k in range(len(aircrafts)):
        for i in range(len(airports)):
            for j in range(len(airports)):
                if(x[i][j][k].varValue!=0):
                    sum = sum+x[i][j][k].varValue
        total_aircrafts.append(sum)
        sum=0
        
    Sol = {'Profit': value(prob.objective), 'Boeing 787':total_aircrafts[0], 'Airbus 320':total_aircrafts[1],
          'Boeing 737':total_aircrafts[2],'Boeing 767':total_aircrafts[3],'Boeing 777':total_aircrafts[4]}
    return(Sol)

        

In [4]:
output = []
for i in range(15):
    output.append(run_prob())
    df = pd.DataFrame(output)

In [5]:
df

,Profit,Boeing 787,Airbus 320,Boeing 737,Boeing 767,Boeing 777
0,3.164340e+06,36.0,86.0,136.0,54.0,0.0
1,2.750111e+06,48.0,97.0,136.0,11.0,19.0
2,3.224570e+06,31.0,97.0,136.0,51.0,0.0
3,2.283515e+06,48.0,97.0,135.0,30.0,2.0
4,3.690923e+06,39.0,97.0,136.0,44.0,0.0
5,2.891199e+06,28.0,97.0,133.0,54.0,0.0
6,3.218223e+06,48.0,97.0,136.0,15.0,15.0
7,2.978391e+06,47.0,95.0,136.0,34.0,0.0
8,3.286093e+06,48.0,97.0,136.0,0.0,36.0
9,1.908645e+06,12.0,90.0,136.0,54.0,19.0


In [7]:
df.nsmallest(16,'Profit')

,Profit,Boeing 787,Airbus 320,Boeing 737,Boeing 767,Boeing 777
9,1.908645e+06,12.0,90.0,136.0,54.0,19.0
12,2.238246e+06,48.0,97.0,136.0,21.0,10.0
3,2.283515e+06,48.0,97.0,135.0,30.0,2.0
13,2.448566e+06,48.0,97.0,134.0,24.0,10.0
1,2.750111e+06,48.0,97.0,136.0,11.0,19.0
10,2.844162e+06,48.0,97.0,113.0,54.0,0.0
5,2.891199e+06,28.0,97.0,133.0,54.0,0.0
7,2.978391e+06,47.0,95.0,136.0,34.0,0.0
0,3.164340e+06,36.0,86.0,136.0,54.0,0.0
6,3.218223e+06,48.0,97.0,136.0,15.0,15.0
